# Hecho servicios acumulados

In [ ]:
from datetime import date
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
import pandas as pd
import numpy as np
import yaml

### Conexión con la base de datos

In [3]:
# Cargar configuraciones
with open('../config.yml', 'r') as f:
  config = yaml.safe_load(f)
  config_oltp = config['fuente']
  config_olap = config['bodega']

# Crear URLs de conexión
url_oltp = (f"{config_oltp['drivername']}://{config_oltp['user']}:{config_oltp['password']}@{config_oltp['host']}:"f"{config_oltp['port']}/{config_oltp['dbname']}")
url_olap = (f"{config_olap['drivername']}://{config_olap['user']}:{config_olap['password']}@{config_olap['host']}:"f"{config_olap['port']}/{config_olap['dbname']}")

# Crear las conexiones
oltp_conn = create_engine(url_oltp)
olap_conn = create_engine(url_olap)

# Función para probar la conexión con las bases de datos
def comprobar_conexionBD(db_engine, db_name):
  try:
    with db_engine.connect() as connection:
      if connection.execute(text("SELECT 1")).fetchone() is not None:
        print(f"Conexión exitosa a base de datos {db_name}")
      else:
        print(f"No se pudo conectar a {db_name}")
  except SQLAlchemyError as e:
    print(f"Error al conectar a base de datos {db_name}: {e}")

comprobar_conexionBD(oltp_conn, config_oltp['dbname'])
comprobar_conexionBD(olap_conn, config_olap['dbname'])


Conexión exitosa a base de datos mensajeriaOLTP
Conexión exitosa a base de datos mensajeriaOLAP


### Módulo de extracción

In [36]:
table_servicio = pd.read_sql_table('mensajeria_servicio', oltp_conn)
table_estadosservicio = pd.read_sql_table('mensajeria_estadosservicio', oltp_conn)
table_estados = pd.read_sql_table('mensajeria_estado', oltp_conn)
dimension_fecha = pd.read_sql('dim_fecha', olap_conn)
dimension_fecha['hora'] = dimension_fecha['hora'].astype(str).str[:5]
hecho_servicios = table_servicio[['id', 'cliente_id', 'mensajero_id', 'ciudad_destino_id', 'ciudad_origen_id', 'mensajero2_id', 'mensajero3_id']]

estados = [
  (1, 'id_fecha_iniciado'),
  (2, 'id_fecha_mensajero'),
  (3, 'id_fecha_recogida'),
  (4, 'id_fecha_entrega'),
  (5, 'id_fecha_terminado')]

### Módulo de transformación

In [37]:
for estado_id, nombre in estados: # genera todas las llaves con la dimensión fecha 
  filtered_estados = (table_estadosservicio[['estado_id', 'servicio_id', 'fecha', 'hora']]
    .query(f"estado_id == {estado_id}")
    .drop(columns=['estado_id'])
    .drop_duplicates(subset=['servicio_id']))
  
  filtered_estados['hora'] = filtered_estados['hora'].astype(str).str[:5]

  filtered_estados = (pd.merge(filtered_estados, dimension_fecha[['fecha', 'hora', 'id']],
    how='left', 
    on=['fecha', 'hora'])
    .rename(columns={'id': nombre})
    .drop(columns=['fecha', 'hora']))

  hecho_servicios = (hecho_servicios
    .merge(filtered_estados, how='left', left_on='id', right_on='servicio_id')
    .drop(columns=['servicio_id'])
    .astype('Int32')
    .replace({pd.NA: None}))

for i in range(len(estados) - 1):
  estado_actual = estados[i][1]
  estado_siguiente = estados[i + 1][1]
    
  tiempos_estados = hecho_servicios[['id', estado_actual, estado_siguiente]].rename(columns={'id': 'id_servicio'})
    
  tiempos_estados = (pd.merge(tiempos_estados, dimension_fecha[['id', 'fecha_hora']], 
    how='left', left_on=estado_actual, right_on='id')
    .rename(columns={'fecha_hora': estado_actual.replace("id_", "")})
    .drop(columns=['id', estado_actual]))

  tiempos_estados = (pd.merge(tiempos_estados, dimension_fecha[['id', 'fecha_hora']], 
    how='left', left_on=estado_siguiente, right_on='id')
    .rename(columns={'fecha_hora': estado_siguiente.replace("id_", "")})
    .drop(columns=['id', estado_siguiente]))

  # diferencia de tiempo entre los pares de estados
  tiempos_estados['diferencia_tiempo'] = tiempos_estados[estado_siguiente.replace("id_", "")] - tiempos_estados[estado_actual.replace("id_", "")]

  # diferencia en horas y minutos
  tiempos_estados[f'horas{estado_actual.replace("id_fecha", "")}_{estado_siguiente.replace("id_fecha_", "")}'] = ((tiempos_estados['diferencia_tiempo'].dt.total_seconds() / 3600)
    .round()
    .astype('Int32')
    .replace({pd.NA: None}))
    
  tiempos_estados.drop(columns=[estado_actual.replace("id_", ""), estado_siguiente.replace("id_", ""), 'diferencia_tiempo'])

  hecho_servicios = (pd.merge(hecho_servicios, tiempos_estados.drop(columns=[estado_actual.replace("id_", ""), estado_siguiente.replace("id_", ""), 'diferencia_tiempo']),
    how='left', left_on='id', right_on='id_servicio')
    .drop(columns=['id_servicio']))

tiempos_estados = hecho_servicios[['id', estados[0][1], estados[len(estados) - 1][1]]]

tiempos_estados = (pd.merge(tiempos_estados.rename(columns={'id' : 'id_servicio'}), dimension_fecha[['id', 'fecha_hora']], 
  how='inner', left_on=estados[0][1], right_on='id')
  .drop(columns=['id_fecha_iniciado', 'id'])
  .rename(columns={'fecha_hora' : 'fecha_hora_iniciado'}))

tiempos_estados = (pd.merge(tiempos_estados, dimension_fecha[['id', 'fecha_hora']], 
  how='inner', left_on='id_fecha_terminado', right_on='id')
  .drop(columns=['id_fecha_terminado', 'id'])
  .rename(columns={'fecha_hora' : 'fecha_hora_terminado'}))

tiempos_estados['horas_iniciado_terminado'] = (
  ((tiempos_estados['fecha_hora_terminado'] - tiempos_estados['fecha_hora_iniciado']).dt.total_seconds() / 3600)
  .round()
  .astype('Int32')
  .replace({pd.NA: None}))

hecho_servicios = (pd.merge(hecho_servicios, tiempos_estados.drop(columns=['fecha_hora_iniciado', 'fecha_hora_terminado']), 
  how='inner', left_on='id', right_on='id_servicio')
  .drop(columns=['id_servicio']))

hecho_servicios.sort_values(by=['id']).head()
#print(hecho_servicios.shape[0])
#tiempos_estados.head(2)

,id,cliente_id,mensajero_id,ciudad_destino_id,ciudad_origen_id,mensajero2_id,mensajero3_id,id_fecha_iniciado,id_fecha_mensajero,id_fecha_recogida,id_fecha_entrega,id_fecha_terminado,horas_iniciado_mensajero,horas_mensajero_recogida,horas_recogida_entrega,horas_entrega_terminado,horas_iniciado_terminado
26849,7,5,1,1,1,7,None,301,624,None,642,643,577,None,None,5,1009
5,8,5,7,1,1,None,None,302,735,None,4779,23045,2212,None,None,1321,4872
26907,10,5,7,1,1,None,None,304,770,None,5799,12814,2403,None,None,514,4145
26848,12,5,9,1,1,7,None,306,618,None,636,637,429,None,None,0,944
26847,13,5,7,1,1,None,None,307,617,None,626,793,428,None,None,1924,2507


### Módulo de carga a la bodega de datos

In [38]:
hecho_servicios.set_index('id', inplace=True)
try:
  hecho_servicios.to_sql('hecho_servicios', olap_conn, if_exists='replace')
except Exception as e:
  print(f"Error al cargar datos: {e}")